In [1]:
%matplotlib inline
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
from tools.common import disp_grid
from IPython.display import Image
from skimage.io import imsave

from keras.models import model_from_json
import os
import joblib
import numpy as np
from lightjob.cli import load_db
#from skimage.io import imread
from tools.common import find_generation_job, find_training_job, to_generation
from IPython.display import Image, display
from lightjob.cli import load_db
from lightjob.db import SUCCESS
import pandas as pd

/home/mcherti/work/code/external/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [ ]:
db = load_db()
jobs = db.jobs_with(state=SUCCESS, where='jobset76')
S = [j['summary'] for j in jobs]
stats = {s: pd.read_csv('../jobs/results/{}/csv/stats.csv'.format(s)) for s in S}

## checking individual job

In [ ]:
job_id = '4d8d586688c602b07cd4ab46f1081295'
job = db.get_job_by_summary(job_id)
folder = '../jobs/results/{}'.format(job_id)
data = joblib.load(os.path.join(folder, 'images.npz'))
data = data[:, -1] # last time step images
model_folder = '../tools/models/mnist/m2'
model = model_from_json(open(os.path.join(model_folder, 'model.json')).read())
model.load_weights(os.path.join(model_folder, 'model.h5'))

In [ ]:
preds = model.predict(data)

In [ ]:
CLASS = 1

In [ ]:
#img = data[ind]
img = data
#print(img.shape)
img = disp_grid(img, border=1, bordercolor=(0.3, 0, 0))

In [ ]:
bins = np.linspace(0, 1, 11)
bin_index = np.digitize(preds[:, CLASS], bins)

In [ ]:
w = data.shape[3] * (len(bins)-1)
h = data.shape[2] * (len(bins)*100)
IMG = np.ones((h, w))*0.5
for i in range(1, len(bins)):
    d = data[bin_index==i]
    p = preds[bin_index==i, CLASS]
    s = np.argsort(p)
    p = p[s]
    p = p[::-1]
    d = d[s]
    d = d[::-1]
    x = (i-1) * data.shape[3]
    y=0
    for im in d:
        IMG[y:y+data.shape[2], x:x+data.shape[3]] = im[0, :, :]
        y+=d.shape[2]
    IMG[:, (i-1) * data.shape[3]]=0.3
    for y in range(len(d)):
        IMG[y*d.shape[2], :]=0.3

In [ ]:
plt.hist(preds[:, CLASS])
plt.xlabel('proba of {}'.format(CLASS))
plt.ylabel('freq')

In [ ]:
job['stats']['out_of_the_box_classification']['tools/models/mnist/m2']['objectness']

In [ ]:
imsave('out.png',IMG)
Image('out.png')

In [ ]:
plt.plot((stats[find_training_job(job_id)['summary']]['train_recons_error'] / (28*28.)).dropna(), label='train')
plt.plot((stats[find_training_job(job_id)['summary']]['test_recons_error'] / (28*28.)).dropna(), label='test')
plt.xlabel('iter')
plt.legend()
plt.ylabel('reconstruction error')

## checking stats of jobs

In [30]:
db = load_db()
jobs = db.jobs_with(state=SUCCESS, where='jobset19')
S = [j['summary'] for j in jobs]

In [31]:
jobs_gen = to_generation(jobs, db=db)

In [34]:
field = 'stats.out_of_the_box_classification.m2.objectness'
indices = np.arange(len(jobs))
for i in range(len(jobs_gen)):
    j = jobs_gen[i]
    v = db.get_value(j, field, if_not_found=None)
    if not v:print(jobs[i]['summary'])

67366f937b2a3788e70c38542be640d0
63e30dbda3a8908e64deeeb7c496a25c
f9750ac8f7e56d25adf22f970cfc5401
18ddfbf10f68d3f892ceafd4248b84dc
2e018add3b456d276fa63cdfe6d5f6c6
d70e85c47052ad467f9250cbf946516f
255fe9680845060becedf9ce7d582089
d537ab4824c3014bc370180dea60d8d8
6befa7992a88144912d4e984f8021d31
29d7d942904f284564074f75f7a1a923
e69fe94c101ace63228b9cf9b77dec04
412d40290d053b6220967bacf9457863
97fdcc0009f4bffeb79fc2398585fe01
76e8c85627b7a0ca6412ac265ab41017
10a649ebae4a215871507c42da2a35a3
390a1b3fad183391bf1a2bef31b757d2
4db775fd7d1f5f5e2b02d420a18ade71
53d2a00139088482b1aa3c144a286f4e


In [36]:
field = 'stats.out_of_the_box_classification.m2.objectness'
indices = np.arange(len(jobs))
objectness = map(lambda j:db.get_value(j, field, if_not_found=None), jobs_gen)
objectness = np.array(objectness)
print(objectness)
indices = sorted(indices, key=lambda i:objectness[i])
indices = filter(lambda ind:objectness[ind], indices)
indices = indices[::-1]
images = ['../exported_data/figs/generated/jobset{:05d}/{}.png'.format(int(jobs[i]['where'][6:]), jobs_gen[i]['summary']) 
          for i in indices]
#for i in range(len(images)):
#    display(Image(images[i]))

[None None nan nan nan nan None None nan nan nan nan None nan None nan nan
 None nan None nan None None nan nan None nan None nan nan nan None nan
 None None nan None None nan None nan nan nan nan]
